## Run

In [ ]:
%%bash
TOP=${TOP:-$(git rev-parse --show-toplevel)}
PATH=${TOP}/build/Examples/Cxx

WSCC_9bus_mult_coupled ${TOP}/Examples/CIM/WSCC-09_RX/*.xml

In [ ]:
%%bash
TOP=${TOP:-$(git rev-parse --show-toplevel)}
PATH=${TOP}/build/Examples/Cxx

WSCC_9bus_mult_decoupled ${TOP}/Examples/CIM/WSCC-09_RX/*.xml

In [ ]:
%%bash
TOP=${TOP:-$(git rev-parse --show-toplevel)}
PATH=${TOP}/build/Examples/Cxx

WSCC_9bus_mult_diakoptics ${TOP}/Examples/CIM/WSCC-09_RX/*.xml

## Validate Results

In [ ]:
import villas.dataprocessing.readtools as rt
from villas.dataprocessing.timeseries import TimeSeries as ts
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [ ]:
path = 'logs/WSCC_9bus_coupled/'
logName = 'WSCC_9bus_coupled'
logFilename = path + logName + '.csv'
print(logFilename)

ts_coupled = rt.read_timeseries_dpsim(logFilename)
phasors_coupled = ts.phasors(ts_coupled)

In [ ]:
path = 'logs/WSCC_9bus_decoupled/'
logName = 'WSCC_9bus_decoupled'
logFilename = path + logName + '.csv'
print(logFilename)

ts_decoupled = rt.read_timeseries_dpsim(logFilename)
phasors_decoupled = ts.phasors(ts_decoupled)

In [ ]:
path = 'logs/WSCC_9bus_diakoptics/'
logName = 'WSCC_9bus_diakoptics'
logFilename = path + logName + '.csv'
print(logFilename)

ts_diakpotics = rt.read_timeseries_dpsim(logFilename)
phasors_diakoptics= ts.phasors(ts_diakpotics)

In [ ]:
for node, phasor in phasors_coupled.items():
    print(node + ': ' + str(phasor['abs'].values[-1]) + '<' + str(phasor['phase'].values[-1]))

In [ ]:
for node, phasor in phasors_decoupled.items():
    print(node + ': ' + str(phasor['abs'].values[-1]) + '<' + str(phasor['phase'].values[-1]))

In [ ]:
for node, phasor in phasors_diakoptics.items():
    print(node + ': ' + str(phasor['abs'].values[-1]) + '<' + str(phasor['phase'].values[-1]))

### Reference - single system

In [ ]:
path = 'logs/WSCC-9bus/'
logName = 'WSCC-9bus'
logFilename = path + logName + '.csv'
print(logFilename)

ts_single = rt.read_timeseries_dpsim(logFilename)
phasors_single = ts.phasors(ts_single)

In [ ]:
for node, phasor in phasors_single.items():
    print(node + ': ' + str(phasor['abs'].values[-1]) + '<' + str(phasor['phase'].values[-1]))

### Comparison

In [ ]:
for node, phasor in phasors_single.items():
    print(node + ': ' + str(phasor['abs'].values[-1] - phasors_decoupled[node]['abs'].values[-1]) + '<' + str(phasor['phase'].values[-1] - phasors_decoupled[node]['phase'].values[-1]))

In [ ]:
for node, phasor in phasors_coupled.items():
    print(node + ': ' + str(phasor['abs'].values[-1] - phasors_diakoptics[node]['abs'].values[-1]) + '<' + str(phasor['phase'].values[-1] - phasors_diakoptics[node]['phase'].values[-1]))

## Performance

In [ ]:
#%%capture
import pandas as pd
from dpsim.MeasurementUtils import Measurement
import numpy as np

logs_dir = '../../../logs_9bus/'
#coupled_meas = []
coupled_mean = []
coupled_max = []
coupled_std = []

for copies in range(0,20):    
    for threads in range(0,13,2):   
        seq_meas = []
        for seq in range(1,11):
            log_path =  logs_dir + 'WSCC_9bus_coupled_' + str(copies) + '_' + str(threads) + '_' + str(seq) + '/' + \
                        'WSCC_9bus_coupled_' + str(copies) + '_' + str(threads) + '_' + str(seq) + '_step_times.log'            
            #print(log_path)
            meas = Measurement.read_timestep_csv(log_path)['step_time'].data
            # aggregate the measurements from the different iterations
            seq_meas = np.concatenate( (seq_meas, meas), axis=0)
            
        #print(seq_meas.shape)
        #coupled_meas.append({'copies': copies, 'threads': threads, 'values': meas['step_time'].data})         
        coupled_mean.append({'copies': copies, 'threads': threads, 'values': seq_meas.mean()})   
        coupled_max.append({'copies': copies, 'threads': threads, 'values': seq_meas.max()})   
        coupled_std.append({'copies': copies, 'threads': threads, 'values': seq_meas.std()})   
        
#pd_coupled_meas = pd.DataFrame(coupled_meas)
pd_coupled_mean = pd.DataFrame(coupled_mean)
pd_coupled_max = pd.DataFrame(coupled_max)
pd_coupled_std = pd.DataFrame(coupled_std)

In [ ]:
import matplotlib.ticker as ticker
import matplotlib.pyplot as plt

ax = plt.figure().gca()
ax.xaxis.set_major_locator(ticker.MultipleLocator(2))
copies = np.arange(1,21)

for threads in range(0,13,2):
    plt.plot(copies, pd_coupled_mean.query('threads=='+str(threads))['values'].values, label=str(threads)+' threads', marker='o')

plt.xlabel('number of system copies')
plt.ylabel('mean timestep duration (s)')
plt.grid()
plt.legend(loc='upper left')
plt.tight_layout()
plt.savefig('9bus_coupled.pdf')

In [ ]:
import pandas as pd
from dpsim.MeasurementUtils import Measurement

logs_dir = '../../../logs_9bus/'
decoupled_meas = []
decoupled_mean = []
decoupled_max = []
decoupled_std = []

for copies in range(0,20):    
    for threads in range(0,13,2):   
        seq_meas = []
        for seq in range(1,11):
            log_path =  logs_dir + 'WSCC_9bus_decoupled_' + str(copies) + '_' + str(threads) + '_' + str(seq) + '/' + \
                        'WSCC_9bus_decoupled_' + str(copies) + '_' + str(threads) + '_' + str(seq) + '_step_times.log'            
            #print(log_path)
            meas = Measurement.read_timestep_csv(log_path)['step_time'].data
            # aggregate the measurements from the different iterations
            seq_meas = np.concatenate( (seq_meas, meas), axis=0)
            
        #print(seq_meas.shape)
        #coupled_meas.append({'copies': copies, 'threads': threads, 'values': meas['step_time'].data})         
        decoupled_mean.append({'copies': copies, 'threads': threads, 'values': seq_meas.mean()})   
        decoupled_max.append({'copies': copies, 'threads': threads, 'values': seq_meas.max()})   
        decoupled_std.append({'copies': copies, 'threads': threads, 'values': seq_meas.std()})   
        
pd_decoupled_meas = pd.DataFrame(decoupled_meas)
pd_decoupled_mean = pd.DataFrame(decoupled_mean)
pd_decoupled_max = pd.DataFrame(decoupled_max)
pd_decoupled_std = pd.DataFrame(decoupled_std)

In [ ]:
import matplotlib.ticker as ticker

ax = plt.figure().gca()
ax.xaxis.set_major_locator(ticker.MultipleLocator(2))
copies = np.arange(1,21)

for threads in range(0,13,2):    
    plt.plot(copies, pd_decoupled_mean.query('threads=='+str(threads))['values'].values, label=str(threads)+' threads', marker='o')

#plt.ylim([0,0.00025])

plt.xlabel('number of system copies')
plt.ylabel('mean timestep duration (s)')
plt.grid()
plt.legend()
plt.tight_layout()
plt.savefig('9bus_decoupled.pdf')

In [ ]:
#%%capture
from dpsim.MeasurementUtils import Measurement 
import numpy as np
import pandas as pd

logs_dir = '../../../logs_9bus/'
diakoptics_meas = []
diakoptics_mean = []
diakoptics_max = []
diakoptics_std = []
            
for copies in range(0,20):    
    for threads in range(0,13,2):   
        for splits in range(0, copies+2):
            seq_meas = []
            for seq in range(1,11):
                log_path =  logs_dir + 'WSCC_9bus_diakoptics_' + str(copies) + '_' + str(threads) + '_' + str(splits) + '_' + str(seq) + '/' + \
                            'WSCC_9bus_diakoptics_' + str(copies) + '_' + str(threads) + '_' + str(splits) + '_' + str(seq) + '_step_times.log'            
                #print(log_path)
                meas = Measurement.read_timestep_csv(log_path)['step_time'].data
                # aggregate the measurements from the different iterations
                seq_meas = np.concatenate( (seq_meas, meas), axis=0)

            #print(seq_meas.shape)
            #diakoptics_meas.append({'copies': copies, 'threads': threads, 'splits': splits, 'values': meas['step_time'].data})         
            diakoptics_mean.append({'copies': copies, 'threads': threads, 'splits': splits, 'values': seq_meas.mean()})   
            diakoptics_max.append({'copies': copies, 'threads': threads, 'splits': splits, 'values': seq_meas.max()})   
            diakoptics_std.append({'copies': copies, 'threads': threads, 'splits': splits, 'values': seq_meas.std()})   
        
#pd_diakoptics_meas = pd.DataFrame(diakoptics_meas)
pd_diakoptics_mean = pd.DataFrame(diakoptics_mean)
pd_diakoptics_max = pd.DataFrame(diakoptics_max)
pd_diakoptics_std = pd.DataFrame(diakoptics_std)

In [ ]:
diakoptics_mean_split_min = []
for copies in range(0,20):    
    for threads in range(0,13,2): 
        split_min_num = copies+1
        split_min = pd_diakoptics_mean.query('copies=='+str(copies)+' and threads=='+str(threads)+' and splits=='+str(split_min_num))['values'].iloc[0]
        for splits in range(1, copies+1):
            tmp = pd_diakoptics_mean.query('copies=='+str(copies)+' and threads=='+str(threads)+' and splits=='+str(splits))['values'].iloc[0]
            if tmp < split_min:
                split_min_num = splits
                split_min = tmp
                
        diakoptics_mean_split_min.append({'copies': copies, 'threads': threads, 'splits': split_min_num, 'values': split_min})  
        
pd_diakoptics_mean_split_min = pd.DataFrame(diakoptics_mean_split_min)

In [ ]:
pd_diakoptics_mean_split_min.query('threads=='+str(8))

In [ ]:
diakoptics_splits = []

for copies in range(0,20):    
    for threads in range(0,13,2):
        diakoptics_splits.append(pd_diakoptics_mean_split_min.query('copies=='+str(copies)+' and threads=='+str(threads))['splits'].values)

In [ ]:
import matplotlib.ticker as ticker

ax = plt.figure().gca()
ax.xaxis.set_major_locator(ticker.MultipleLocator(2))
copies = np.arange(1,21)

for threads in range(0,13,2):
    plt.plot(copies, pd_diakoptics_mean_split_min.query('threads=='+str(threads))['splits'].values, label= str(threads) + ' threads', marker='o')


plt.xlabel('number of system copies')
plt.ylabel('mean split number')
plt.grid()
plt.legend()
plt.tight_layout()
plt.savefig('diakoptics_min_mean_splits.pdf')

In [ ]:
import matplotlib.ticker as ticker

ax = plt.figure().gca()
ax.xaxis.set_major_locator(ticker.MultipleLocator(2))
copies = np.arange(1,21)

for threads in range(0,13,2):
    plt.plot(copies, pd_diakoptics_mean_split_min.query('threads=='+str(threads))['values'].values, label= str(threads) + ' threads', marker='o')

plt.xlabel('number of system copies')
plt.ylabel('mean timestep duration (s)')
plt.grid()
plt.legend()
plt.tight_layout()
plt.savefig('9bus_diakoptics.pdf')

In [ ]:
import matplotlib.ticker as ticker

ax = plt.figure().gca()
ax.xaxis.set_major_locator(ticker.MultipleLocator(2))
copies = np.arange(1,21)
threads = 8

plt.plot(copies, pd_coupled_mean.query('threads=='+str(threads))['values'].values, label='coupled, 8 threads', marker='o')
plt.plot(copies, pd_decoupled_mean.query('threads=='+str(threads))['values'].values, label='line decoupled, 8 threads', marker='o')
plt.plot(copies, pd_diakoptics_mean_split_min.query('threads=='+str(threads))['values'].values, label='diakoptics, 8 threads', marker='o')

plt.xlabel('number of system copies')
plt.ylabel('mean timestep duration (s)')
plt.grid()
plt.legend()
plt.yscale('log')
plt.tight_layout()
plt.savefig('9bus_comparison.pdf')
plt.savefig('9bus_comparison.png', dpi=300)